### Batch Normalization / 批样本单通道  
feature map：包含 N 个样本，每个样本通道数为 C，高为 H，宽为 W。对其求均值和方差时，将在 N、H、W上操作，而保留通道 C 的维度。       
   
具体来说，就是把第1个样本的第1个通道，加上第2个样本第1个通道 ...... 加上第 N 个样本第1个通道，求平均，得到通道 1 的均值（注意是除以 N×H×W 而不是单纯除以 N，最后得到的是一个代表这个 batch 第1个通道平均值的数字，而不是一个 H×W 的矩阵）。求通道 1 的方差也是同理。对所有通道都施加一遍这个操作，就得到了所有通道的均值和方差

In [26]:
import torch
import torch.nn as nn

In [28]:
# BN，批样本单通道。
# LN，单样本。
# Instance，单样本单通道。
# GN，单样本批通道

In [32]:
# track_running_stats=False，求当前 batch 真实平均值和标准差，而不是更新全局平均值和标准差
# affine=False, 只做归一化，不乘以 gamma 加 beta（通过训练才能确定）
# num_features 为 feature map 的 channel 数目
# eps 设为 0，让官方代码和我们自己的代码结果尽量接近
bn = nn.BatchNorm2d(num_features=3, eps=0, affine=False, track_running_stats=False)

x = torch.randn(10, 3, 5, 5) * 10000
x.shape, x.is_contiguous()

(torch.Size([10, 3, 5, 5]), True)

In [34]:
bn_x = bn(x)
print(bn_x.shape)

x1 = x.permute(1, 0, 2, 3).reshape(3, -1)
m = x1.mean(dim=1).view(1, 3, 1, 1)
print(x1.shape, m.shape)

# unbiased=False, 求方差时不做无偏估计（除以 N-1 而不是 N），和原始论文一致
std = x1.std(dim=1, unbiased=False).view(1, 3, 1, 1)
my_bn = (x - m) / std
print(std.shape, my_bn.shape)

diff = (bn_x - my_bn).sum()
print('diff={}'.format(diff)) # 差别是 10-5 级的，证明和官方版本基本一致

torch.Size([10, 3, 5, 5])
torch.Size([3, 250]) torch.Size([1, 3, 1, 1])
torch.Size([1, 3, 1, 1]) torch.Size([10, 3, 5, 5])
diff=3.6665587686002254e-06


### 2. Group Nomalization / 单样本批通道     
GN 计算均值和标准差时，把每一个样本 feature map 的 channel 分成 G 组，每组将有 C/G 个 channel，然后将这些 channel 中的元素求均值和标准差。各组 channel 用其对应的归一化参数独立地归一化

In [16]:
x = torch.randn(10, 20, 5, 5) * 10000
gn = nn.GroupNorm(num_groups=4, num_channels=20, eps=0, affine=False)
gn_x = gn(x)

In [24]:
x1 = x.view(10, 4, -1)
m = x1.mean(dim=-1).reshape(10, 4, -1)
std = x1.std(dim=-1).reshape(10, 4, -1)
print(x1.shape, m.shape)

x1_norm = (x1 - m) / std
my_gn = x1.reshape(10, 20, 5, 5)
print(x1_norm.shape)

diff = (gn_x - my_gn).sum()
print('diff={}'.format(diff))

torch.Size([10, 4, 125]) torch.Size([10, 4, 1])
torch.Size([10, 4, 125])
diff=276564.90625
